In [269]:
import torch
import numpy as np
import torch.nn as nn
import sys
import os
project_root = os.path.abspath("..")  # Adjust if needed
import pytorch_lightning as pl
# Add the project root to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

from src.models.pointNetVae import PointNetVAE
from src.utils.data_utils import *
from src.dataset_classes.pointDataset import *
from proteinshake.datasets import ProteinFamilyDataset
from proteinshake.tasks import LigandAffinityTask
import random
from torch.utils.data import DataLoader, Dataset, Subset
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [250]:
optimizer = torch.optim.AdamW
optimizer_param = {'lr':0.001}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = PointNetVAE(2, optimizer, optimizer_param)

In [270]:
from proteinshake.transforms import CenterTransform
dataset = ProteinFamilyDataset(root='../data').to_point().torch()

In [277]:
idx_list = range(len(dataset))
subset_size = int(len(dataset)//10)
val_idx = random.sample(idx_list, subset_size)  # Get random subset
train_idx = list(set(idx_list) - set(val_idx))
s = 500
train_subset = PointDataset(Subset(dataset, train_idx), s)
val_subset = PointDataset(Subset(dataset, val_idx), s)

100%|██████████| 3110/3110 [00:00<00:00, 11901.89it/s]


In [ ]:
latent_dim = 128
epochs = 30
lr = 0.0001
batch_size = 256
train_dataloader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_subset,batch_size=batch_size, shuffle=False)
x_dim = train_subset[0].shape[0]
if torch.cuda.is_available():
    torch.cuda.current_device()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [316]:
import torch.optim.adam
model = PointNetVAE(latent_dim,torch.optim.Adam,{'lr':0.0001}, beta=1,embed_dim = 64, global_feature_size = 1024)
model


PointNetVAE(
  (input_t_net): T_NET(
    (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
    (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    (conv3): Conv1d(128, 512, kernel_size=(1,), stride=(1,))
    (fc1): Linear(in_features=512, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=64, bias=True)
    (fc3): Linear(in_features=64, out_features=9, bias=True)
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (max_pool): MaxPool1d(kernel_size=500, stride=500, padding=0, dilation=1, ceil_mode=False)
    (relu): ReLU()
  )
  (feature_t_net): T

In [317]:
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping
import pytorch_lightning as pl
optimizer = torch.optim.Adam
optimizer_param = {'lr':0.001}
trainer = pl.Trainer(max_epochs=epochs,
    accelerator="auto",
    devices="auto",
    logger=TensorBoardLogger(save_dir="logs/"))

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [318]:
trainer.fit(model, train_dataloader, val_dataloader)


   | Name            | Type        | Params | Mode 
---------------------------------------------------------
0  | input_t_net     | T_NET       | 150 K  | train
1  | feature_t_net   | T_NET       | 420 K  | train
2  | embedding       | Embedding   | 1.3 K  | train
3  | shared_conv1d_1 | Conv1d      | 256    | train
4  | shared_conv1d_2 | Conv1d      | 4.2 K  | train
5  | shared_conv1d_3 | Conv1d      | 4.2 K  | train
6  | shared_conv1d_4 | Conv1d      | 8.3 K  | train
7  | shared_conv1d_5 | Conv1d      | 132 K  | train
8  | bn1             | BatchNorm1d | 128    | train
9  | bn2             | BatchNorm1d | 128    | train
10 | bn3             | BatchNorm1d | 128    | train
11 | bn4             | BatchNorm1d | 256    | train
12 | bn5             | BatchNorm1d | 2.0 K  | train
13 | fc1_enc_mu      | Linear      | 131 K  | train
14 | fc1_enc_logvar  | Linear      | 131 K  | train
15 | relu            | ReLU        | 0      | train
16 | max_pool        | MaxPool1d   | 0      | train
17 | 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined